In [ ]:
from srai.loaders.osm_loaders import OSMPbfLoader
from srai.regionalizers import H3Regionalizer
from srai.joiners import IntersectionJoiner
from srai.embedders import ContextualCountEmbedder
from srai.plotting.folium_wrapper import plot_regions, plot_numeric_data
from srai.neighbourhoods import H3Neighbourhood

## Data preparation

In order to use `ContextualCountEmbedder` we need to prepare some data.  
Namely we need: `regions_gdf`, `features_gdf`, and `joint_gdf`.  
These are the outputs of Regionalizers, Loaders and Joiners respectively.

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

area_gdf = geocode_to_region_gdf("Lisboa, PT")
plot_regions(area_gdf)

### Regionalize the area using an H3Regionalizer

In [ ]:
regionalizer = H3Regionalizer(resolution=9, buffer=True)
regions_gdf = regionalizer.transform(area_gdf)
regions_gdf

### Download some objects from OpenStreetMap

You can use both `OsmTagsFilter` and `GroupedOsmTagsFilter` filters. In this example, a predefined `GroupedOsmTagsFilter` filter `BASE_OSM_GROUPS_FILTER` is used.

In [ ]:
from srai.loaders.osm_loaders.filters import BASE_OSM_GROUPS_FILTER

loader = OSMPbfLoader()
features_gdf = loader.load(area_gdf, tags=BASE_OSM_GROUPS_FILTER)
features_gdf

### Join the objects with the regions they belong to

In [ ]:
joiner = IntersectionJoiner()
joint_gdf = joiner.transform(regions_gdf, features_gdf)
joint_gdf

## Embed using features existing in data

`ContextualCountEmbedder` extends capabilities of basic `CountEmbedder` by incorporating the neighbourhood of embedded region. In this example we will use the `H3Neighbourhood`.

In [ ]:
h3n = H3Neighbourhood()

### Squashed vector version (default)

Embedder will return vector of the same length as `CountEmbedder`, but will sum averaged values from the neighbourhoods diminished by the neighbour distance squared.

In [ ]:
cce = ContextualCountEmbedder(
    neighbourhood=h3n, neighbourhood_distance=10, concatenate_vectors=False
)
embeddings = cce.transform(regions_gdf, features_gdf, joint_gdf)
embeddings

### Concatenated vector version

Embedder will return vector of length `n * distance` where `n` is number of features from the `CountEmbedder` and `distance` is number of neighbourhoods analysed.

Each feature will be postfixed with `_n` string, where `n` is the current distance. Values are averaged from all neighbours.

In [ ]:
wide_cce = ContextualCountEmbedder(
    neighbourhood=h3n, neighbourhood_distance=10, concatenate_vectors=True
)
wide_embeddings = wide_cce.transform(regions_gdf, features_gdf, joint_gdf)
wide_embeddings

### Plotting example features

In [ ]:
plot_numeric_data(regions_gdf, embeddings, "leisure", tiles_style="CartoDB positron")

In [ ]:
plot_numeric_data(regions_gdf, embeddings, "transportation", tiles_style="CartoDB positron")